<a href="https://colab.research.google.com/github/Deepashree8/DEEPA813/blob/main/DL_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install transformers datasets torch scikit-learn

In [9]:
from transformers import BertTokenizer,BertForSequenceClassification
from transformers import Trainer,TrainingArguments
import torch

In [10]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from datasets import load_dataset
dataset=load_dataset('imdb')
def tokenize_function(examples):
  return tokenizer(examples['text'],padding="max_length",truncation=True,max_length=256)
tokenized_dataset=dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

4.Split the datasets

In [14]:
train_dataset=tokenized_dataset['train'].shuffle(seed=42).select(range(1000)) #sub_sample for quick training
test_dataset=tokenized_dataset['test'].shuffle(seed=42).select(range(200))

5.Training_arguments

In [16]:
training_args=TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

6.Define Trainer

In [17]:
from transformers import Trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

7.Train model

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.608600,0.366360
2,0.282900,0.384239
3,0.250000,0.494431


TrainOutput(global_step=375, training_loss=0.4038123590151469, metrics={'train_runtime': 166.5307, 'train_samples_per_second': 18.015, 'train_steps_per_second': 2.252, 'total_flos': 394666583040000.0, 'train_loss': 0.4038123590151469, 'epoch': 3.0})

8.Evaluate model

In [19]:
trainer.evaluate()

{'eval_loss': 0.4944314658641815,
 'eval_runtime': 2.9235,
 'eval_samples_per_second': 68.411,
 'eval_steps_per_second': 4.447,
 'epoch': 3.0}

In [22]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

9.Make prediction

In [24]:
text="The movie was not fantastic"
inputs=tokenizer(text,return_tensors="pt",truncation=True,padding=True).to(device)
with torch.no_grad():
  outputs=model(**inputs)
  logits=outputs.logits
  predicted_class=torch.argmax(logits,dim=-1).item()
print(f"predicted class:{predicted_class}")

predicted class:0
